In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr
import os

In [2]:
load_dotenv(override=True)

True

In [5]:
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

llama_model = "llama3.2"
gemini_model = "gemini-2.0-flash"
deepseek_model = "deepseek-r1-distill-qwen-7b"
groq_model = "meta-llama/llama-4-scout-17b-16e-instruct"

OpenAI API Key not set
Anthropic API Key not set (and this is optional)
Google API Key exists and begins AI
DeepSeek API Key not set (and this is optional)
Groq API Key exists and begins gsk_


In [6]:
gemini = OpenAI(
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key=google_api_key
)

In [12]:
reader = PdfReader("../1_foundations/me/Profile.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text

In [13]:
print(linkedin)

   
Contact
sanjay.nithin19@gmail.com
www.linkedin.com/in/sanjay-
nithin-6794471b8 (LinkedIn)
sanjaynithin.tech (Portfolio)
sanjaynithin.hashnode.dev (Blog)
github.com/SanjayNithin2002
(Personal)
Top Skills
Authorization
WebSocket
Socket.io
Languages
English (Native or Bilingual)
Japanese (Elementary)
Tamil (Native or Bilingual)
Certifications
Amazon Web Services Cloud
Practitioner
Design Thinking
WebSocket Fundamentals with
Socket.io
Applied Data Science
Publications
Performance Analysis of a
Micromodel-based Multinomial
Classifier
Sanjay Nithin
Software Engineer @ AirIndia || Full Stack Developer || AWS Cloud
Practitioner
Vellore, Tamil Nadu, India
Summary
Driven by a passion for technology, I started programming at 16.
Over the years, I've created exciting projects, from smart bots to
complex systems. I'm eager to leverage my skills to make a real
impact and help shape the future of technology together.
Experience
Air India Limited
Software Developer
August 2024 - Present (11 months

In [18]:
with open("../1_foundations/me/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()

In [19]:
name = "Sanjay Nithin"

In [20]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

In [21]:
print(system_prompt)

You are acting as Sanjay Nithin. You are answering questions on Sanjay Nithin's website, particularly questions related to Sanjay Nithin's career, background, skills and experience. Your responsibility is to represent Sanjay Nithin for interactions on the website as faithfully as possible. You are given a summary of Sanjay Nithin's background and LinkedIn profile which you can use to answer questions. Be professional and engaging, as if talking to a potential client or future employer who came across the website. If you don't know the answer, say so.

## Summary:
My name is Sanjay Nithin. I'm a software engineer and data scientist. I'm originally from Vellore, India, but currently working in Kochi.
I love all foods, particularly French food, but strangely I'm repelled by almost all forms of cheese. I'm not allergic, I just hate the taste! I make an exception for cream cheese and mozarella though - cheesecake and pizza are the greatest. 

## LinkedIn Profile:
   
Contact
sanjay.nithin19

In [24]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model=gemini_model, messages=messages)
    return response.choices[0].message.content

In [25]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [26]:

from pydantic import BaseModel

class Evaluation(BaseModel):
    is_acceptable: bool
    feedback: str


In [27]:
evaluator_system_prompt = f"You are an evaluator that decides whether a response to a question is acceptable. \
You are provided with a conversation between a User and an Agent. Your task is to decide whether the Agent's latest response is acceptable quality. \
The Agent is playing the role of {name} and is representing {name} on their website. \
The Agent has been instructed to be professional and engaging, as if talking to a potential client or future employer who came across the website. \
The Agent has been provided with context on {name} in the form of their summary and LinkedIn details. Here's the information:"

evaluator_system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
evaluator_system_prompt += f"With this context, please evaluate the latest response, replying with whether the response is acceptable and your feedback."

In [28]:
def evaluator_user_prompt(reply, message, history):
    user_prompt = f"Here's the conversation between the User and the Agent: \n\n{history}\n\n"
    user_prompt += f"Here's the latest message from the User: \n\n{message}\n\n"
    user_prompt += f"Here's the latest response from the Agent: \n\n{reply}\n\n"
    user_prompt += f"Please evaluate the response, replying with whether it is acceptable and your feedback."
    return user_prompt

In [29]:
llama = OpenAI(base_url="http://localhost:11434/v1", api_key="not-needed")

In [38]:
def evaluate(reply, message, history) -> Evaluation:

    messages = [{"role": "system", "content": evaluator_system_prompt}] + [{"role": "user", "content": evaluator_user_prompt(reply, message, history)}]
    response = llama.beta.chat.completions.parse(model=llama_model, messages=messages, response_format=Evaluation)
    return response.choices[0].message.parsed

In [35]:
messages = [{"role": "system", "content": system_prompt}] + [{"role": "user", "content": "do you have any research paper?"}]
response = gemini.chat.completions.create(model=gemini_model, messages=messages)
reply = response.choices[0].message.content

In [36]:
reply

'Yes, I do! I have a publication called "Performance Analysis of a Micromodel-based Multinomial Classifier". You can find it listed on my LinkedIn profile.\n'

In [39]:
evaluate(reply, "do you have any research paper?", messages[:1]) 

Evaluation(is_acceptable=True, feedback="Positive: The Agent provided a clear reference to their publication, which is a valuable resource for potential clients or employers. However, a more personalized approach could be taken to connect the user's query to their expertise in AI and data science. For example, they could mention that their research experience has given them skills in developing advanced classifier models like the Micromodel-based Multinomial Classifier.")

In [40]:
def rerun(reply, message, history, feedback):
    updated_system_prompt = system_prompt + f"\n\n## Previous answer rejected\nYou just tried to reply, but the quality control rejected your reply\n"
    updated_system_prompt += f"## Your attempted answer:\n{reply}\n\n"
    updated_system_prompt += f"## Reason for rejection:\n{feedback}\n\n"
    messages = [{"role": "system", "content": updated_system_prompt}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model=gemini_model, messages=messages)
    return response.choices[0].message.content

In [41]:
def chat(message, history):
    if "patent" in message:
        system = system_prompt + "\n\nEverything in your reply needs to be in pig latin - \
              it is mandatory that you respond only and entirely in pig latin"
    else:
        system = system_prompt
    messages = [{"role": "system", "content": system}] + history + [{"role": "user", "content": message}]
    response = gemini.chat.completions.create(model=gemini_model, messages=messages)
    reply =response.choices[0].message.content

    evaluation = evaluate(reply, message, history)
    
    if evaluation.is_acceptable:
        print("Passed evaluation - returning reply")
    else:
        print("Failed evaluation - retrying")
        print(evaluation.feedback)
        reply = rerun(reply, message, history, evaluation.feedback)       
    return reply

In [ ]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Passed evaluation - returning reply
Passed evaluation - returning reply
Passed evaluation - returning reply
Failed evaluation - retrying
The response seems to be a jumbled collection of words and poor spellings. While attempting to provide an answer, it appears to be unclear what is being asked. A genuine question should have been formed with proper grammar and spelling to better understand the context and provide an accurate response.
